In [1]:
!pip install numpy
!pip install pandas
!pip install nashpy

  Created wheel for nashpy: filename=nashpy-0.0.21-py3-none-any.whl size=15280 sha256=c7bdf38cf77dc89a3c0005ca26135ec4ad2ce21c22def93f16fe288af39033a2
  Stored in directory: /root/.cache/pip/wheels/02/08/62/cf4fa931e0a317d180936b266169a57f4bb4eb801465bbe8a1
Successfully built nashpy


In [2]:
!pip install data

  Created wheel for data: filename=data-0.4-py3-none-any.whl size=7247 sha256=941c8cd887f8da6fbee818e65bf37f9eac8b8e68fe05cd3b7be1c9c6ae8be003
  Stored in directory: /root/.cache/pip/wheels/28/76/4b/7db27e02c1de4d2181559d108fcdfed1680ff7d3534553d460
Successfully built data


In [3]:
import random
import nashpy as nash
import numpy as np
import data as dt
import time
import copy 
import networkx as nx 
import matplotlib.pyplot as plt
from math import e
from sklearn import preprocessing
import seaborn as sns
from decimal import Decimal
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

data = []

In [4]:

def generate_graph(graph_type, nodes, payoffs):
  graph = nx.empty_graph(nodes, create_using=nx.DiGraph())
  if(graph_type == "er"):
    graph = nx.erdos_renyi_graph(nodes, 0.5)
  elif(graph_type == "ws"):
    graph = nx.watts_strogatz_graph(nodes, 3, 0.5)
  for n in graph.nodes():
    x = random.randrange(2)
    y = random.randrange(2)
    p = payoffs[x][y]
    graph.nodes[n]['weight'] = p
  return graph 
  

In [5]:
def calc_transitions(p_matrix, graph):
    p = (p_matrix[1][1] * p_matrix[0][1]) / (p_matrix[0][0] + p_matrix[1][1] - p_matrix[0][1] - p_matrix[1][0])
    w_pos = 0
    w_neg = 0
    pos_arr = []
    neg_arr = []
    s1 = 0 
    s2 = 0 
    for n in graph.nodes: 
       k_deg = graph.degree[n] 
       if(graph.nodes[n]['weight'] == p_matrix[0][0] or graph.nodes[n]['weight'] == p_matrix[1][0]):
         s1 = s1 + 1
       else:
         s2 = s2 + 1
       uk_neg_prime = k_deg * ((p*p_matrix[1][0]) + (1-p)*p_matrix[1][1])
       uk_neg_reg = k_deg * ((p*p_matrix[0][0]) + (1 - p)*p_matrix[0][1])
       uk_pos_prime = k_deg * ((p*p_matrix[0][0]) + (1 - p)*p_matrix[0][1])
       uk_pos_reg = k_deg * ((p*p_matrix[1][0]) + (1-p)*p_matrix[1][1])
       step1 = (uk_pos_reg - uk_pos_prime)/0.1
       step2 = (uk_neg_reg - uk_neg_prime)/0.1
       w_pos = 1/(1 + (e ** step1))
       w_neg = 1/(1 + (e ** step2))
       trans_pos = s1*w_pos
       pos_arr.append(trans_pos)
       trans_neg = s1*w_neg 
       neg_arr.append(trans_neg)
      
    pos_arr = np.array(pos_arr)
    neg_arr = np.array(neg_arr)
    normalized_pos = (pos_arr - pos_arr.min())/ (pos_arr.max() - pos_arr.min())
    normalized_neg = (neg_arr - neg_arr.min())/ (neg_arr.max() - neg_arr.min())
    normalized_pos.reshape(1, -1)
    normalized_neg.reshape(1, -1)

    return np.array(normalized_pos), np.array(normalized_neg)


In [6]:
def calc_average(array1):
  sum1 = 0
  avg1 = 0
  for i in array1:
    sum1 = sum1 + i
  avg1 = sum1/len(array1)
  return avg1  

In [7]:
def fixation_dist(a1):
  sum1 = 0
  sum2 = 0 
  for i in range(len(a1)):
    prod = 1
    for j in range(i):
      trans_prob = a1[i]
      prod = prod * trans_prob
    sum1 = sum1 + prod
  return 1/(1 + sum1)
  


In [8]:
def monte_carlo(payoffs):
  pos = []
  neg = []
  transition1 = 0 
  transition2 = 0 
  fin1 = []
  fin2 = []
  fin3 = []
  fin4 = []
  results1 = 0 
  results2 = 0 
  list1 = []
  list2 = []
  for i in range(10000):
    graph1 = generate_graph("er", 100, payoffs)
    graph2 = generate_graph("ws", 100, payoffs)

    pos1, neg1 = calc_transitions(payoffs, graph1)
    np.concatenate((pos1, neg1), axis = 0)


    pos2, neg2 = calc_transitions(payoffs, graph2)
    np.concatenate((pos2, neg2), axis = 0)

    transition1 = fixation_dist(pos1)
    results1 = results1 + transition1
    prob1 = results1/(i + 1)
    list1.append(prob1)
    fin1.append(transition1)

    transition2 = fixation_dist(pos2)
    results2 = results2 + transition2
    prob2 = results2/(i+1)
    list2.append(prob2)
    fin2.append(transition2)
  
  return fin1, fin2, list1, list2
  

In [12]:

payoffs1 = np.array([[1.2, 0], [0.8, 1]])
payoffs2 = np.array([[1.25, 0.05],[0.85, 1.05]])
payoffs3 = np.array([[1.3, 0.1], [0.9, 1.1]])
payoffs4 = np.array([[1.35, 0.15], [0.95, 1.15]])
payoffs5 = np.array([[1.40, 0.20], [1, 1.20]])

payoffs6 = np.array([[2.2, 1], [1.8, 2]])
payoffs7 = np.array([[2.3, 1.1], [1.9, 2.1]])
payoffs8 = np.array([[2.4, 1.2], [2.0, 2.2]])
payoffs9 = np.array([[2.5, 1.3], [2.1, 2.3]])
payoffs10 = np.array([[2.6, 1.4], [2.2, 2.4]])
 
fin1, fin2, list1, list2 = monte_carlo(payoffs1)
fin3, fin4, list3, list4 = monte_carlo(payoffs2)
fin5, fin6, list5, list6 = monte_carlo(payoffs3)
fin7, fin8, list7, list8 = monte_carlo(payoffs4)
fin9, fin10, list9, list10 = monte_carlo(payoffs5)

fin11, fin12,list11, list12  = monte_carlo(payoffs6)
fin13, fin14, list13, list14 = monte_carlo(payoffs7)
fin15, fin16, list15, list16 = monte_carlo(payoffs8)
fin17, fin18, list17, list18 = monte_carlo(payoffs9)
fin19, fin20, list19, list20 = monte_carlo(payoffs10)

plt.show()

fig, axes = plt.subplots(1, 2, constrained_layout = False)

fig = sns.kdeplot(fin1, color = "r", ax = axes[0])
fig = sns.kdeplot(fin3, color = "b", ax = axes[0])
fig = sns.kdeplot(fin5, color = "g", ax = axes[0])
fig = sns.kdeplot(fin7, color = "y", ax = axes[0])
fig = sns.kdeplot(fin9, color = "m", ax = axes[0])

fig = sns.kdeplot(fin11, color = "r", ax = axes[0])
fig = sns.kdeplot(fin13, color = "b", ax = axes[0])
fig = sns.kdeplot(fin15, color = "g", ax = axes[0])
fig = sns.kdeplot(fin17, color = "y", ax = axes[0])
fig = sns.kdeplot(fin19, color = "m", ax = axes[0])

plt.legend()

fig = sns.kdeplot(fin2, color = "r", ax = axes[1])
fig = sns.kdeplot(fin4, color = "b", ax = axes[1])
fig = sns.kdeplot(fin6, color = "g", ax = axes[1])
fig = sns.kdeplot(fin8, color = "y", ax = axes[1])
fig = sns.kdeplot(fin10, color = "m", ax = axes[1])

fig = sns.kdeplot(fin12, color = "r", ax = axes[1])
fig = sns.kdeplot(fin14, color = "b", ax = axes[1])
fig = sns.kdeplot(fin16, color = "g", ax = axes[1])
fig = sns.kdeplot(fin18, color = "y", ax = axes[1])
fig = sns.kdeplot(fin20, color = "m", ax = axes[1])
 
fig.figure.suptitle("Fixation Distributions", fontsize=18)

fig, axs = plt.subplots(5, 4)

axs[0, 0].plot(list1)
axs[0, 1].plot(list2)
axs[0, 2].plot(list3)
axs[0, 3].plot(list4)
axs[1, 0].plot(list6)
axs[2, 0].plot(list7)
axs[3, 0].plot(list8)
axs[4, 0].plot(list9)
axs[1, 1].plot(list10)
axs[1, 2].plot(list11)
axs[1, 3].plot(list12)
axs[2, 1].plot(list13)
axs[2, 2].plot(list14)
axs[2, 3].plot(list15)
axs[3, 1].plot(list16)
axs[3, 2].plot(list17)
axs[3, 3].plot(list18)
axs[4, 1].plot(list19)
axs[4, 2].plot(list20)



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: RuntimeWarning: overflow encountered in double_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: RuntimeWarning: overflow encountered in double_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:40: RuntimeWarning: invalid value encountered in true_divide


KeyboardInterrupt: ignored